In [5]:
# !pip install keras==2.2.0

In [6]:
# !pip install tensorflow-estimator==1.10.12

In [7]:
# !pip install tensorflow-serving-api==1.12.0

In [8]:
# !pip install tensorflow==1.12.0

In [9]:
 # !pip freeze

In [1]:
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D,Conv2D
from keras.models import Model
from keras.optimizers import RMSprop,SGD
from keras.preprocessing.image import ImageDataGenerator
import time
from IPython.display import display 
#from PIL import Image
from keras.utils import multi_gpu_model
import tensorflow as tf
import keras
from PIL import Image, ImageFilter
from tensorflow.keras import backend as k


Using TensorFlow backend.


In [2]:
config = tf.ConfigProto()
 
tf.keras.backend.set_session(tf.Session(config=config))
starttime = time.time()

# dimensions of our images.
img_width, img_height = 60,320#720, 1280#180, 320

#train_data_dir = 'C:/train'
train_data_dir = 'pickle/'
#validation_data_dir = 'C:/test'
validation_data_dir = 'pickle/'
nb_train_samples = 10#70000/100
nb_validation_samples = 100
nb_epoch = 1
batch_size = 100

input_img = Input(shape=( img_width, img_height,3))

#with tf.device('/gpu:0'):
x = Conv2D(16, 3, 3, activation='relu', border_mode='same')(input_img)
print(x)
x = MaxPooling2D((2, 2), border_mode='same')(x)
print(x)
x = Conv2D(8, 3, 3, activation='relu', border_mode='same')(x)
#x = MaxPooling2D((2, 2), border_mode='same')(x)
#x = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(x)
encoded = MaxPooling2D((2, 2), border_mode='same')(x)

# at this point the representation is (8, 4, 4) i.e. 128-dimensional

x = Conv2D(8, 3, 3, activation='relu', border_mode='same')(encoded)
x = UpSampling2D((2, 2))(x)
#x = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(x)
#x = UpSampling2D((2, 2))(x)
x = Conv2D(16, 3, 3, activation='relu', border_mode='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, 3, 3, activation='sigmoid', border_mode='same')(x)

#with tf.device('/cpu:0'):
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')




Tensor("conv2d_1/Relu:0", shape=(?, 60, 320, 16), dtype=float32)

Tensor("max_pooling2d_1/MaxPool:0", shape=(?, 30, 160, 16), dtype=float32)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", padding="same")`
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:23: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((2, 2), padding="same")`
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), activation="relu", padding="same")`
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:28: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((2, 2), padding="same")`
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:32: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8

In [3]:
from keras.models import model_from_json
import numpy as np
import os
import cv2
import skimage 
from skimage import io
from skimage.transform import rescale, resize, downscale_local_mean
import pickle as pk


def generate_arrays_from_file(flag):
    if flag == True:
        path = train_data_dir
    else:
        path = validation_data_dir
        
    imagesName = os.listdir(path)
    print(imagesName)
    print("*************************generate_arrays_from_file")
    while(1):
        count = 1
        for image in imagesName:
            filename = path+"/"+image
            
            file = open(filename,"rb")
            data = pk.load(file)
            print(data.shape)
            data = data.reshape(data.shape[0],60,320,3)
            print(data.shape)
            count =count + 1
            print("Count:",count)
            yield(data,data)
#with tf.device("/gpu:0"):
            
autoencoder.fit_generator(
    generate_arrays_from_file(0),
    samples_per_epoch=nb_train_samples,
    nb_epoch=nb_epoch,
    validation_data=generate_arrays_from_file(0),
    nb_val_samples=nb_validation_samples
    )    

print("time...", time.time()-starttime)

model_json = autoencoder.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
autoencoder.save_weights("model.h5")
print("Saved model to disk")


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:40: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:40: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., steps_per_epoch=10, epochs=1, validation_steps=100)`




Epoch 1/1
['pickle2.pk', 'pickle1.pk', 'pickle0.pk']
*************************generate_arrays_from_file
['pickle2.pk', 'pickle1.pk', 'pickle0.pk']
*************************generate_arrays_from_file
(100, 57600)
(100, 60, 320, 3)
Count: 2
(100, 57600)
(100, 60, 320, 3)
Count: 2



(100, 57600)
(100, 60, 320, 3)
Count: 3
(100, 57600)
(100, 60, 320, 3)
Count: 3
(100, 57600)
(100, 57600)
(100, 60, 320, 3)
Count: 4
(100, 60, 320, 3)
Count: 4

(100, 57600)
(100, 60, 320, 3)
Count: 2
(100, 57600)
(100, 60, 320, 3)
Count: 2
(100, 57600)
(100, 57600)
(100, 60, 320, 3)
Count: (100, 60, 320, 3)
Count: 3
3
(100, 57600)
(100, 60, 320, 3)
Count: 4
(100, 57600)
(100, 60, 320, 3)
Count: 4
(100, 57600)
(100, 60, 320, 3)
Count: 2
(100, 57600)
(100, 60, 320, 3)
Count: 2
(100, 57600)
(100, 60, 320, 3)
Count: 3
(100, 57600)
(100, 60, 320, 3)
Count: 3
(100, 57600)
(100, 60, 320, 3)
Count: 4
(100, 57600)
(100, 60, 320, 3)
Count: 4
(100, 57600)
(100, 60, 320, 3)
Count: 2
(100, 57600)
(100, 60, 320, 3)
Count